In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! unzip '/content/drive/MyDrive/Dataset/Dataset.zip'

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load dataset
train_data_dir = '/content/Dataset/Train'
validation_data_dir = '/content/Dataset/Valid'
test_data_dir = '/content/Dataset/Test'
img_height, img_width = 224, 224
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

# Load pre-trained VGG model without top (fully connected layers)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add new top layers
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(2, activation='softmax')(x)

# Create new model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size)

# Evaluate the model
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

loss, accuracy = model.evaluate(test_generator)
print("Test Accuracy:", accuracy)

Found 10002 images belonging to 2 classes.
Found 10002 images belonging to 2 classes.
58889256/58889256 [==============================] - 4s 0us/step


Epoch 1/10
312/312 [==============================] - 179s 565ms/step - loss: 0.7561 - accuracy: 0.7016 - val_loss: 0.3886 - val_accuracy: 0.8512
Epoch 2/10
312/312 [==============================] - 180s 577ms/step - loss: 0.5052 - accuracy: 0.7448 - val_loss: 0.3185 - val_accuracy: 0.8686
Epoch 3/10
312/312 [==============================] - 179s 574ms/step - loss: 0.5001 - accuracy: 0.7523 - val_loss: 0.3178 - val_accuracy: 0.8715
Epoch 4/10
312/312 [==============================] - 182s 582ms/step - loss: 0.4892 - accuracy: 0.7595 - val_loss: 0.3529 - val_accuracy: 0.8682
Epoch 5/10
312/312 [==============================] - 181s 581ms/step - loss: 0.4794 - accuracy: 0.7699 - val_loss: 0.3009 - val_accuracy: 0.8728
Epoch 6/10
312/312 [==============================] - 180s 577ms/step - loss: 0.4867 - accuracy: 0.7588 - val_loss: 0.3147 - val_accuracy: 0.8606
Epoch 7/10
312/312 [==============================] - 178s 571ms/step - loss: 0.4571 - accuracy: 0.7787 - val_loss: 0.2998 -

In [ ]:
y_pred_prob = model.predict(test_generator)
y_pred = np.argmax(y_pred_prob, axis=1)  # Convert probabilities to class labels

# Get true labels
true_labels = test_generator.classes

# Generate classification report
class_names = list(test_generator.class_indices.keys())
print("Classification Report on Test Data:")
print(classification_report(true_labels, y_pred, target_names=class_names,zero_division=0))

313/313 [==============================] - 38s 121ms/step
Classification Report on Test Data:
              precision    recall  f1-score   support

        Fake       0.79      0.64      0.71      5001
        Real       0.70      0.83      0.76      5001

    accuracy                           0.74     10002
   macro avg       0.74      0.74      0.73     10002
weighted avg       0.74      0.74      0.73     10002



In [ ]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(true_labels, y_pred)
print(conf_matrix)


[[3223 1778]
 [ 856 4145]]


In [ ]:
from tensorflow.keras.applications import EfficientNetB0

In [ ]:
# Load pre-trained EfficientNet model
base_model_E = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add new top layers
x = Flatten()(base_model_E.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(2, activation='softmax')(x)

# Create new model
model_E = Model(inputs=base_model_E.input, outputs=output)

# Compile the model
model_E.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model_E.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size)

# Evaluate the model
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

loss, accuracy = model_E.evaluate(test_generator)
print("Test Accuracy:", accuracy)

16705208/16705208 [==============================] - 2s 0us/step


Epoch 1/10
312/312 [==============================] - 221s 587ms/step - loss: 0.5595 - accuracy: 0.8473 - val_loss: 1.0800 - val_accuracy: 0.5499
Epoch 2/10
312/312 [==============================] - 193s 617ms/step - loss: 0.2219 - accuracy: 0.9194 - val_loss: 0.6053 - val_accuracy: 0.6840
Epoch 3/10
312/312 [==============================] - 179s 572ms/step - loss: 0.1724 - accuracy: 0.9397 - val_loss: 0.7545 - val_accuracy: 0.4995
Epoch 4/10
312/312 [==============================] - 181s 578ms/step - loss: 0.1494 - accuracy: 0.9492 - val_loss: 0.8003 - val_accuracy: 0.5318
Epoch 5/10
312/312 [==============================] - 179s 574ms/step - loss: 0.1705 - accuracy: 0.9456 - val_loss: 2.6013 - val_accuracy: 0.5063
Epoch 6/10
312/312 [==============================] - 177s 566ms/step - loss: 0.1404 - accuracy: 0.9538 - val_loss: 1.0567 - val_accuracy: 0.4995
Epoch 7/10
312/312 [==============================] - 178s 569ms/step - loss: 0.1200 - accuracy: 0.9593 - val_loss: 73.7901 

In [ ]:
y_pred_prob = model_E.predict(test_generator)
y_pred = np.argmax(y_pred_prob, axis=1)  # Convert probabilities to class labels

# Get true labels
true_labels = test_generator.classes

# Generate classification report
class_names = list(test_generator.class_indices.keys())
print("Classification Report on Test Data:")
print(classification_report(true_labels, y_pred, target_names=class_names,zero_division=0))

313/313 [==============================] - 21s 64ms/step
Classification Report on Test Data:
              precision    recall  f1-score   support

        Fake       0.69      0.44      0.54      5001
        Real       0.59      0.80      0.68      5001

    accuracy                           0.62     10002
   macro avg       0.64      0.62      0.61     10002
weighted avg       0.64      0.62      0.61     10002



In [ ]:
conf_matrix = confusion_matrix(true_labels, y_pred)
print(conf_matrix)

[[2202 2799]
 [ 999 4002]]


In [ ]:
model.save('vgg.h5')
model_E.save('eff.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
